### ResNet Implementation

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader , Dataset , random_split
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.notebook import tqdm , trange

In [3]:
num_epochs = 5
batch_size = 64
learning_rate = 1e-3


model_name = "Custom_ResNet"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cpu


In [4]:
# Creating Transformations for the input data

transform = transforms.Compose([transforms.Resize(32) , transforms.ToTensor() , transforms.Normalize((0.5 , 0.5 , 0.5) , (0.5 , 0.5 , 0.5))])

In [8]:
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


validation_split = 0.1

validation_num = int(len(train_dataset) * validation_split)
training_num = len(train_dataset) - validation_num

train_dataset , val_dataset = random_split(train_dataset , [training_num , validation_num] )

In [ ]:
# A Convolutional Block with no residual connection
class ConvoBlock(nn.Module):
    def __init__(self , channels_in):
        super().__init__()
        # We wan't to half the number of channels in the input
        self.conv1 = nn.Conv2d(in_channels=channels_in , out_channels=channels_in//2 , kernel_size=3 , padding=1 , stride=1)
        self.batchnorm1 = nn.BatchNorm2d(num_features=channels_in//2)
        self.conv2 = nn.Conv2d(in_channels=channels_in//2 , out_channels=channels_in , kernel_size=3 , padding=1 , stride=1)

    def forward(self , x):
        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.conv2(x)
        return x
    

# A Residual Block with skip connection
class ResnetBlock(nn.Module):
    def __init__(self , channels_in):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=channels_in , out_channels=channels_in//2 , kernel_size=3 , padding=1 , stride=1)
        self.batchnorm1 = nn.BatchNorm2d(num_features=channels_in//2)
        self.conv2 = nn.Conv2d(in_channels=channels_in//2 , out_channels=channels_in , kernel_size=3 , padding=1 , stride=1)


    def forward(self , x):
        identity = x

        # Activation before skip connection
        x = F.relu(self.conv1(x))

        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.conv2(x)

        # Skip connection
        x = x + identity
        return x
        

# A Residual Block that can change the number of channels in the input
class ResDownBlock(nn.Module):
    def __init__(self , channel_in , channel_out=None):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=channel_in , out_channels=channel_out , kernel_size=3 , padding=1 , stride=1)
        self.batchnorm1 = nn.BatchNorm2d(num_features=channel_out)
        self.conv2 = nn.Conv2d(in_channels=channel_out , out_channels=channel_out , kernel_size=3 , padding=1 , stride=1)


        self.conv3 = nn.Conv2d(in_channels=channel_in , out_channels=channel_out , kernel_size=3 , padding=1 , stride=1)

        if channel_out is None:
            channel_out = channel_in
        

    def forward(self , x):
        identity = self.conv3(x)

        x = F.relu(self.batchnorm1(self.conv1(x)))
        x = self.conv2(x)

        x = x + identity

        return x

# A Block with skip connection that concatenates the outputs of "raw" layers
class SkipBlock(nn.Module):
        def __init__(self, channels):
            super().__init__()
            
            self.conv1 = nn.Conv2d(channels, channels//2,  kernel_size=3, stride=1, padding=1)
            self.bn1 = nn.BatchNorm2d(channels//2)
            self.conv2 = nn.Conv2d(channels//2, channels//2,  kernel_size=3, stride=1, padding=1)
        
        def forward(self, x):
            x = F.relu(x)
            
            x1 = self.conv1(x)
            x2 = F.relu(self.bn1(x1))
            x3 = self.conv2(x2)
            
            # Skip concatenation of "raw" layer outputs
            return torch.cat((x1, x3), 1)


class Deep_CNN(nn.Module):
    def __init__(self , num_classes=10):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3 , out_channels=64 , kernel_size=3 , padding=1 , stride=1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2 , stride=2)
        )

        self.layer2 = nn.Sequential(
            ResnetBlock(64),
            ResnetBlock(64),
            ResDownBlock(64 , 128),
            nn.MaxPool2d(kernel_size=2 , stride=2)
        )

        self.layer3 = nn.Sequential(
            ResnetBlock(128),
            ResnetBlock(128),
            ResDownBlock(128 , 256),
            nn.MaxPool2d(kernel_size=2 , stride=2)
        )

        self.layer4 = nn.Sequential(
            ResnetBlock(256),
            ResnetBlock(256),
            ResDownBlock(256 , 512),
            nn.MaxPool2d(kernel_size=2 , stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512 , out_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256 , out_features=num_classes)
        )


    def forward(self , x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.classifier(x)

        return x